# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_files/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,5.93,96,97,10.71,TF,2023-12-13
1,1,puerto natales,-51.7236,-72.4875,10.25,43,40,13.38,CL,2023-12-13
2,2,atafu village,-8.5421,-172.5159,26.31,91,100,6.10,TK,2023-12-13
3,3,badger,64.8000,-147.5333,-14.07,84,100,2.57,US,2023-12-13
4,4,alice springs,-23.7000,133.8833,26.75,32,16,2.57,AU,2023-12-13


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
plot_A = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)
# Display the map
plot_A

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,puerto natales,-51.7236,-72.4875,10.25,43,40,13.38,CL,2023-12-13
62,62,punta arenas,-53.1500,-70.9167,10.06,50,75,8.75,CL,2023-12-13
122,122,ancud,-41.8697,-73.8203,11.29,63,9,0.48,CL,2023-12-13
150,150,longyearbyen,78.2186,15.6401,2.91,65,75,7.20,SJ,2023-12-13
280,280,tateyama,34.9833,139.8667,11.72,57,79,1.56,JP,2023-12-13


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,puerto natales,CL,-51.7236,-72.4875,43,
62,punta arenas,CL,-53.1500,-70.9167,50,
122,ancud,CL,-41.8697,-73.8203,63,
150,longyearbyen,SJ,78.2186,15.6401,65,
280,tateyama,JP,34.9833,139.8667,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 1000
params = {'categories':'accommodation.hotel',
         'filter':'',
         'bias':'',
         'apiKey':geoapify_key,
         'limit':1}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon=row['Lng']
    lat=row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
punta arenas - nearest hotel: Hotel Lacolet
ancud - nearest hotel: Cabañas
longyearbyen - nearest hotel: The Vault
tateyama - nearest hotel: No hotel found
gilgit - nearest hotel: PTDC Motel
kamogawa - nearest hotel: 鴨川ユニバースホテル
hadano - nearest hotel: グランドホテル神奈中・秦野
lethbridge - nearest hotel: Sandman Hotel
hasaki - nearest hotel: 大新旅館
medicine hat - nearest hotel: No hotel found
el calafate - nearest hotel: Hostería Las Avutardas
bryne - nearest hotel: Jæren hotell
semirom - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,puerto natales,CL,-51.7236,-72.4875,43,Dorotea Patagonia Hostel
62,punta arenas,CL,-53.1500,-70.9167,50,Hotel Lacolet
122,ancud,CL,-41.8697,-73.8203,63,Cabañas
150,longyearbyen,SJ,78.2186,15.6401,65,The Vault
280,tateyama,JP,34.9833,139.8667,57,No hotel found
364,gilgit,PK,35.9221,74.3087,47,PTDC Motel
402,kamogawa,JP,35.1000,140.1000,61,鴨川ユニバースホテル
409,hadano,JP,35.3711,139.2236,66,グランドホテル神奈中・秦野
441,lethbridge,CA,49.7000,-112.8186,44,Sandman Hotel
497,hasaki,JP,35.7333,140.8333,46,大新旅館


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
plot_B = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)
# Display the map
plot_B

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)